# Get more fancy pants data
- go to the web factset 
- request it
- back to your sheet and run it (insert the name)


# Integrate it with the model
- join the data
- so update the model features


In [6]:
import json
import pandas as pd
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType, DateType
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.registry import model_registry
from snowflake.ml._internal.utils import identifier

# 1.1 Get the Data Share


In [ ]:
https://app.snowflake.com/marketplace/listing/GZT1ZA3NLF/similarweb-ltd-global-stocks-25-000-tickers-digital-traffic-data-by-domain?search=Global%20Stocks

# 1.2 Reading Snowflake Connection Details, create a Session


In [7]:
snowflake_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/creds.json").read()) # <--- Update here
session = Session.builder.configs(snowflake_connection_cfg).create()
session.sql("USE DATABASE HOL_DEMO").collect()
#session.sql("USE STAGE YOUR_STAGE").collect()
session.sql("CREATE OR REPLACE WAREHOUSE ASYNC_WH WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'").collect()

[Row(status='Warehouse ASYNC_WH successfully created.')]

# 1.3 Examine a sample of the data

In [ ]:
>>> df.group_by().agg(sum_("b")).collect()

In [9]:
from snowflake.snowpark.functions import col, lit, sum as sum_, max as max_


In [11]:
sdf = session.sql("""Select * FROM TICK_HISTORY.PUBLIC.TH_SF_MKTPLACE where ticker IN ('AMZN', 'FDS', 'IBM', 'META')""")
sdf_test = sdf.group_by("TICKER").agg(sum_("LAST_VOL"))
sdf_test.limit(5).to_pandas()

,TICKER,SUM(LAST_VOL)
0,FDS,59005626
1,AMZN,7732354508
2,META,3870265028
3,IBM,1028604713


# Use Native Snowpark to Manipulate the Data

spark = SparkSession.builder.appName("example").getOrCreate()
sdf_test = sdf.groupBy("TICKER").agg(sum_("LAST_VOL"))



In [ ]:
snowpark group by stock/day,
max price min price mean price, rebuild ML model

In [ ]:
1	AMZN
2	FDS
3	IBM
4	META